In [1]:
from sklearn.decomposition import NMF
import numpy as np
from scipy.io import mmread

In [3]:
X = mmread('data/data_matrix.mtx').T

In [4]:
X.shape

(99620, 3142)

In [5]:
k = 40

In [6]:
def find_projection(w):
    k = np.shape(w)[0]
    # project onto row-sum-1 space
    wP = w - np.mean(w) + 1/k
    
    def obj_tmp(wSort, lam):
        x = wSort - lam
        return np.sum(x[x>0])
    
    def find_mu(u):
        # binary search for the left and right ind
        # left: f(x) > 1; right: f(x) <1
        k = np.shape(u)[0]
        left, right = 0, k-1
        while right - left > 1:
            mid = (left+right)//2
            f = obj_tmp(wSort, wSort[mid])
            if f == 1:
                return wSort[mid]
            elif f > 1:
                left = mid
            else:
                right = mid
        x1 = wSort[left]
        x2 = wSort[right]
        y1 = obj_tmp(wSort, x1)
        y2 = obj_tmp(wSort, x2)
        return (x2-x1)/(y2-y1) * (1-y2) + x2
    
    # solve for the threshold mu
    wSort = np.sort(wP)
    if wSort[0] >= 0:
        return wP
    else:
        mu = find_mu(wSort)       
        
        # get the solution
        a = wP - mu
        a[a < 0] = 0
        return a

In [8]:
nmf = NMF(n_components=k, init="random", verbose=True)
W = nmf.fit_transform(X, y=None, W=None, H=None)
for j in range(W.shape[0]):
    W[j, :] = find_projection(W[j, :])

violation: 1.0
violation: 0.5153361691530283
violation: 0.24239410250342822
violation: 0.16287447397664176
violation: 0.12562075283136487
violation: 0.10227087363505859
violation: 0.0860970016432616
violation: 0.07459585565528888
violation: 0.0657911952427035
violation: 0.05851844162593436
violation: 0.052550138560716546
violation: 0.04757272657949507
violation: 0.04333613523639311
violation: 0.03966567646438219
violation: 0.03644065475786466
violation: 0.03350348463831211
violation: 0.031057744852544294
violation: 0.028943349627642438
violation: 0.02699443742728248
violation: 0.025343030418169286
violation: 0.023982451083810738
violation: 0.022809406535766085
violation: 0.021790132346294414
violation: 0.020852106147073408
violation: 0.020044202168299913
violation: 0.01932116207982132
violation: 0.01865349848511577
violation: 0.018031245621716017
violation: 0.01748776184148924
violation: 0.01697902220907117
violation: 0.01648598598849042
violation: 0.01603811460181892
violation: 0.0156

/home/yuel5/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [9]:
p = X.shape[1]
Ht = np.zeros(shape=(p, k))

In [11]:
from sklearn.decomposition._nmf import _update_coordinate_descent
from sklearn.utils.validation import check_random_state
random_state = 42
rng = check_random_state(random_state)
def evaluate_obj(X, W, Ht):
    return np.linalg.norm(X - W @ Ht.T)**2/X.shape[0]
def update_H(X, W, Ht):
    _update_coordinate_descent(X.T, Ht, W, 0, 0, shuffle=False, random_state=rng)
    return
def update_W(X, W, Ht):
    n = X.shape[0]
    _update_coordinate_descent(X, W, Ht, 0, 0, shuffle=False, random_state=rng)
    for j in range(n):
        W[j, :] = find_projection(W[j, :])
    return
for i in range(10):
    update_H(X, W, Ht)
    update_W(X, W, Ht)
    print(evaluate_obj(X, W, Ht))

1162.5956666843742
1077.5202427761203
1049.4927656012221
1038.6877314294372
1032.0158665093218
1027.1203003825888
1024.1582748783999
1022.0175262437148
1020.2114136326281
1018.6506676132809


In [13]:
np.save('data/init_W.npy', W)
np.save('data/init_H.npy', Ht.T)